# IMPORTING LIBRARIES


In [2]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import mysql.connector
import sqlite3
# from sklearn.preprocessing import OrdinalEncoder

In [3]:
lst= []
# creating a list which we will use in the scrape function.

In [5]:
pg = 1
url = f'https://books.toscrape.com/catalogue/page-{pg}.html'
# the base url which is defined to scrape
r =requests.get(url,'lxml')
soup = bs(r.text,'lxml')
#  here we used beautiful soup to get all the text for the page


In [6]:
soup.find('a')

<a href="../index.html">Books to Scrape</a>

In [7]:
# soup.prettify()

In [77]:
page= soup.find_all('article',class_= 'product_pod')
#  we filter out the extra data which was defined in the soup variable

In [78]:
# checking the ratings for page 1 
#  we use for loop to iterate in page in class star rating.
for rating in page:
    print(rating.find('p',class_= 'star-rating').get('class')[1])

Three
One
One
Four
Five
One
Four
Three
Four
One
Two
Four
Five
Five
Five
Three
One
One
Two
Two


In [9]:
# for availability in page 1 , we check class instock availibility using for loop  
for stock in page:
    print(stock.find('p', class_='instock availability').text.strip())



In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock
In stock


In [10]:
#  price for page 1 using for loop
for price in page:
    print(price.find('p',class_='price_color').text[1:])

£51.77
£53.74
£50.10
£47.82
£54.23
£22.65
£33.34
£17.93
£22.60
£52.15
£13.99
£20.66
£17.46
£52.29
£35.02
£57.25
£23.88
£37.59
£51.33
£45.17



# Iterating for page 1



# checking all the values and using variables to store it in



#  ex - title,rating ,price,availibility and printing it to check for page 1

In [11]:
# page[0].find('a').get('href')[10:-16]
# title found here
for book in page:
#     print(page.find('a').get('href')[10:-15]) 
    title= book.find('img').get('alt')
    ratings= book.find('p',class_= 'star-rating').get('class')[1]
    price = book.find('p',class_='price_color').text[2:]
    availability  = book.find('p', class_='instock availability').text.strip()
    print(ratings,price,availability,title) 


Three 51.77 In stock A Light in the Attic
One 53.74 In stock Tipping the Velvet
One 50.10 In stock Soumission
Four 47.82 In stock Sharp Objects
Five 54.23 In stock Sapiens: A Brief History of Humankind
One 22.65 In stock The Requiem Red
Four 33.34 In stock The Dirty Little Secrets of Getting Your Dream Job
Three 17.93 In stock The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Four 22.60 In stock The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
One 52.15 In stock The Black Maria
Two 13.99 In stock Starving Hearts (Triangular Trade Trilogy, #1)
Four 20.66 In stock Shakespeare's Sonnets
Five 17.46 In stock Set Me Free
Five 52.29 In stock Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Five 35.02 In stock Rip it Up and Start Again
Three 57.25 In stock Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
One 23.88 In stock Olio
One 37.59 In stock Mesaerion: The Best Scie

"The Upper Code Works Well  for 1 Page"

## Implementing Scrape Function

In [80]:
# main code 
def scrape(url,lst):
    
    pg =1
    soup = bs(r.text,'lxml')
#     soup as we implemented earlier at the start of code

    page= soup.find_all('article',class_= 'product_pod')
#     filtering the page in page variable

    for book in page: 
        title= book.find('img').get('alt')
        ratings= book.find('p',class_= 'star-rating').get('class')[1]
        price = book.find('p',class_='price_color').text[2:]
        availability  = book.find('p', class_='instock availability').text.strip()
#  iterated for page using book 
# title and other variables contain the data for 20 items and then iterate for next page
        lst.append({'title': title,
                'Price':price,
                'Ratings': ratings,
                'Availability':availability
               }# all the details in page 1
              )
#     appending title and other variables in list(lst) to store the data 

while (pg < 51):
    scrape(url,lst)
    pg+=1
    data = pd.DataFrame(lst)
# we have 50 pages so we iterage for 50 pages here 
# and as soon as 51 hits it will stop and list will have all the elements in data 


In [81]:
data.count()
# checking all the values of data

title           2000
Price           2000
Ratings         2000
Availability    2000
dtype: int64

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         2000 non-null   object
 1   Price         2000 non-null   object
 2   Ratings       2000 non-null   object
 3   Availability  2000 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB


In [84]:
data['Ratings']= data['Ratings'].replace(['One','Two',"Three","Four","Five"],[1,2,3,4,5])
# converting ratings into numerical data
# which is originally categorical data
data
# checking data
# all the values are as it should be

,title,Price,Ratings,Availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.10,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock
...,...,...,...,...
1995,Our Band Could Be Your Life: Scenes from the A...,57.25,3,In stock
1996,Olio,23.88,1,In stock
1997,Mesaerion: The Best Science Fiction Stories 18...,37.59,1,In stock
1998,Libertarianism for Beginners,51.33,2,In stock


# Creating Database and Storing Values

In [85]:
# books_schema = """

#     CREATE TABLE books_scraped( 
#         title     TEXT, 
#         Price     REAL, 
#         Rate      INTEGER,
#         in_stock  TEXT,     
#     )
# """
#  create table query

In [25]:
# db = mysql.connector.connect(host='localhost',user= 'root',database= 'scrape_db',password= "Kaushik.9871")

# CREATING A CONNECTION WITH DATABASE

In [88]:
conn = sqlite3.connect('scrape_db')
# creating a connector here with sqlite with dat

In [89]:
c= conn.cursor()
# cursor is created here 
# cursor allows user to  name or access stored data 
c.execute("""CREATE TABLE books(title TEXT, Price REAL , RATINGS REAL,Availability TEXT)""")
# executes sql statements 

OperationalError: table books already exists

In [67]:
conn= create_engine('sqlite:///scrape_db.sqlite', echo=False)
# The SQLAlchemy engine creates a common interface to the database to execute SQL statements.
#  we transfer data from jupyter to sql using engine

In [69]:
print(conn)
# connection succeded

Engine(sqlite:///scrape_db.sqlite)


In [90]:
# data to sql which is used to transfer data as it is

In [70]:
data.to_sql('books', con=conn, if_exists='append', index=False)

In [71]:
query = """
    SELECT * FROM books
"""

In [ ]:
# we use query to read for sql table

In [72]:
df = pd.read_sql_query(query, conn)
# here we stored sql query  data for reading sql table 

In [93]:
df

,title,Price,Ratings,Availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.10,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock
...,...,...,...,...
995,Our Band Could Be Your Life: Scenes from the A...,57.25,3,In stock
996,Olio,23.88,1,In stock
997,Mesaerion: The Best Science Fiction Stories 18...,37.59,1,In stock
998,Libertarianism for Beginners,51.33,2,In stock


# Thank you!